# IaC-GPT Training on Kaggle

Train a domain-specific Infrastructure-as-Code LLM for free on Kaggle GPUs.

**Setup:**
1. Enable GPU: Settings → Accelerator → GPU T4 x2
2. Enable Internet: Settings → Internet → On
3. Run all cells

**Estimated time:** d12 ~3hrs, d16 ~8hrs, d20 ~18hrs

In [ ]:
# === CONFIGURATION ===
# Change these settings based on your available Kaggle hours

MODEL_DEPTH = 12        # 12 (~300M, ~3hrs) | 16 (~500M, ~8hrs) | 20 (~800M, ~18hrs)
BATCH_SIZE = 8          # Reduce to 4 if you get OOM errors
NUM_GPUS = 2            # Kaggle T4 x2

print(f"Training config: d{MODEL_DEPTH} model, batch_size={BATCH_SIZE}, gpus={NUM_GPUS}")

In [ ]:
# Clone the repo
!git clone https://github.com/holynakamoto/iacgpt.git
%cd iacgpt

# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q tiktoken pyarrow filelock rustbpe wandb tabulate regex zstandard

# Install flash-attn (optional, may fail)
!pip install -q flash-attn --no-build-isolation 2>/dev/null || echo "Flash attention not available"

## 2. Collect IaC Training Data

In [ ]:
# Scrape IaC repositories (takes ~10-15 min)
!bash dev/fast_scrape_iac.sh <<< 'n'

In [ ]:
# Convert to training shards
!python dev/repackage_iac_data.py \
    --input-dir data/iac_raw_cloned \
    --output-dir ~/.cache/nanochat/iac_data \
    --include-synthetic --include-docs

In [ ]:
# Setup data directories
!cp ~/.cache/nanochat/iac_data/shard_00000.parquet ~/.cache/nanochat/iac_data/shard_00001.parquet
!ln -sf ~/.cache/nanochat/iac_data ~/.cache/nanochat/base_data
!ls -la ~/.cache/nanochat/base_data/

## 3. Train Custom Tokenizer

In [ ]:
# Train BPE tokenizer on IaC data (~1 min)
!python -m scripts.tok_train

In [ ]:
# Evaluate tokenizer compression
!python -m scripts.tok_eval

## 4. Train IaC-GPT Base Model

In [ ]:
# Train IaC-GPT model
import os
os.environ['OMP_NUM_THREADS'] = '1'

cmd = f"""torchrun --standalone --nproc_per_node={NUM_GPUS} -m scripts.base_train -- \
    --depth={MODEL_DEPTH} \
    --device-batch-size={BATCH_SIZE} \
    --target-param-data-ratio=5 \
    --run=dummy \
    --model-tag=iac-gpt-d{MODEL_DEPTH} \
    --eval-every=200 \
    --sample-every=500 \
    --save-every=1000"""

print(f"Running: {cmd}")
!{cmd}

## 5. Evaluate Base Model

In [ ]:
# Evaluate base model
cmd = f"torchrun --standalone --nproc_per_node={NUM_GPUS} -m scripts.base_eval -- --device-batch-size={BATCH_SIZE}"
print(f"Running: {cmd}")
!{cmd}

## 6. SFT: Add DevOps Architect Persona

In [ ]:
# Generate IaC-specific identity conversations
import json
import os

conversations = [
    {'messages': [
        {'role': 'user', 'content': 'Who are you?'},
        {'role': 'assistant', 'content': 'I am IaC-GPT, a specialized AI assistant for Infrastructure-as-Code. I can help you write Terraform modules, Kubernetes manifests, Ansible playbooks, and Crossplane compositions.'}
    ]},
    {'messages': [
        {'role': 'user', 'content': 'Write a Terraform module for an AWS VPC'},
        {'role': 'assistant', 'content': 'Here is a Terraform module for an AWS VPC:\n\n```hcl\nvariable "vpc_cidr" {\n  default = "10.0.0.0/16"\n}\n\nresource "aws_vpc" "main" {\n  cidr_block = var.vpc_cidr\n  enable_dns_hostnames = true\n  tags = { Name = "main-vpc" }\n}\n```'}
    ]},
    {'messages': [
        {'role': 'user', 'content': 'Create a Kubernetes deployment'},
        {'role': 'assistant', 'content': 'Here is a Kubernetes Deployment:\n\n```yaml\napiVersion: apps/v1\nkind: Deployment\nmetadata:\n  name: app\nspec:\n  replicas: 3\n  selector:\n    matchLabels:\n      app: app\n  template:\n    metadata:\n      labels:\n        app: app\n    spec:\n      containers:\n      - name: app\n        image: nginx:alpine\n        ports:\n        - containerPort: 80\n```'}
    ]},
]

os.makedirs(os.path.expanduser('~/.cache/nanochat'), exist_ok=True)
with open(os.path.expanduser('~/.cache/nanochat/iac_identity_conversations.jsonl'), 'w') as f:
    for _ in range(50):
        for conv in conversations:
            f.write(json.dumps(conv) + '\n')
print('Generated identity conversations')

In [ ]:
# Run SFT
cmd = f"torchrun --standalone --nproc_per_node={NUM_GPUS} -m scripts.chat_sft -- --device-batch-size={BATCH_SIZE} --run=dummy"
print(f"Running: {cmd}")
!{cmd}

## 7. Test Your Model

In [ ]:
!python -m scripts.chat_cli -p "Write a Terraform module for an EKS cluster with 3 node groups"

In [ ]:
!python -m scripts.chat_cli -p "Create a Kubernetes deployment with resource limits and health checks"

In [ ]:
!python -m scripts.chat_cli -p "Review this for security issues: resource aws_s3_bucket data { acl = public-read }"

## 8. Download Model Checkpoint

Save your trained model before the Kaggle session ends!

In [ ]:
# Zip the model checkpoint
!zip -r iac-gpt-model.zip ~/.cache/nanochat/checkpoints/ ~/.cache/nanochat/tokenizer/
print("Download from Kaggle Output tab")